In [ ]:
import os
Root = "/content/drive/MyDrive/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [ ]:
ls

Actor_01/  Actor_05/  Actor_09/  Actor_13/  Actor_17/  Actor_21/
Actor_02/  Actor_06/  Actor_10/  Actor_14/  Actor_18/  Actor_22/
Actor_03/  Actor_07/  Actor_11/  Actor_15/  Actor_19/  Actor_23/
Actor_04/  Actor_08/  Actor_12/  Actor_16/  Actor_20/  Actor_24/


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [43]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [44]:
# Emotions in the dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [45]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [46]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [47]:
x_train

array([[-6.29695435e+02,  5.06518898e+01,  1.60058804e+01, ...,
         1.01923375e-04,  4.90204402e-05,  2.31723443e-05],
       [-6.35375183e+02,  5.90106773e+01,  9.59572554e-01, ...,
         4.37377457e-06,  3.63577965e-06,  1.08047118e-06],
       [-5.59764954e+02,  4.44761124e+01, -2.95931005e+00, ...,
         2.93061021e-04,  2.40781461e-04,  8.95909907e-05],
       ...,
       [-8.03163330e+02,  3.14002113e+01,  1.85828054e+00, ...,
         4.17585397e-06,  4.26851511e-06,  2.89372474e-06],
       [-6.95444519e+02,  6.84528809e+01,  2.32682247e+01, ...,
         8.18211629e-06,  3.46105412e-06,  1.28334534e-06],
       [-5.23901550e+02,  3.90003548e+01, -1.22465200e+01, ...,
         1.16796487e-04,  6.61305530e-05,  5.32570557e-05]])

In [48]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [49]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [50]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [51]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [52]:
#Predict for the test set
y_pred=model.predict(x_test)

In [53]:
y_pred

array(['happy', 'calm', 'fearful', 'calm', 'calm', 'happy', 'calm',
       'calm', 'fearful', 'fearful', 'calm', 'happy', 'disgust', 'happy',
       'calm', 'happy', 'calm', 'happy', 'fearful', 'fearful', 'happy',
       'calm', 'disgust', 'happy', 'calm', 'happy', 'calm', 'fearful',
       'disgust', 'disgust', 'calm', 'happy', 'fearful', 'fearful',
       'disgust', 'fearful', 'calm', 'fearful', 'fearful', 'disgust',
       'fearful', 'disgust', 'fearful', 'happy', 'calm', 'calm',
       'fearful', 'fearful', 'happy', 'disgust', 'happy', 'fearful',
       'calm', 'disgust', 'fearful', 'disgust', 'happy', 'calm',
       'fearful', 'happy', 'disgust', 'disgust', 'fearful', 'happy',
       'happy', 'fearful', 'calm', 'calm', 'fearful', 'happy', 'happy',
       'disgust', 'disgust', 'fearful', 'disgust', 'happy', 'calm',
       'calm', 'calm', 'happy', 'calm', 'calm', 'fearful', 'calm',
       'fearful', 'disgust', 'disgust', 'happy', 'fearful', 'fearful',
       'happy', 'happy', 'disgu

In [54]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 72.92%


In [55]:
from sklearn.metrics import accuracy_score, f1_score

In [56]:
f1_score(y_test, y_pred,average=None)

array([0.8411215 , 0.7       , 0.71578947, 0.64705882])

In [57]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,disgust,calm
2,fearful,fearful
3,calm,calm
4,calm,calm
5,happy,happy
6,calm,calm
7,calm,calm
8,fearful,fearful
9,fearful,fearful


In [58]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [59]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['happy'], dtype='<U7')

In [60]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450531e-01,
         1.43299656e+01,  3.33637071e+00, -2.54071975e+00,
        -4.05790949e+00, -1.07119999e+01, -7.29413939e+00,
         1.74018872e+00, -4.19064236e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803197e+00, -5.68717539e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875393e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902557e-01, -1.65101993e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821292e+00,
        -5.54154456e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990698e+00,
        -3.62910867e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345614e-01,
         6.60463989e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921539e-01,  7.23420441e-01,  7.16119647e-0